# Коллокации и Pointwise Mutual Information (PMI)

**Коллокация** - сочетание из двух или более слов, которое:

1. обладает некомпозициональной семантикой.
2. состоит из двух слов, которые вместе значимо часто.

При лингвистическом анализе текста коллокации нам очень нужны по понятным причинам:

* выявить, о чем текст
* найти различия между текстами на схожую тематику
* находить упоминания каких-то событий и персон
* и т.д.

Значит, нам нужно уметь выделять их автоматически. Чтобы найти коллокации, состоящие из двух слов, нам нужно выделить такие биграммы, которые встречаются друг с другом значимо часто, по сравнению с их индивидуальными частотами.

Для выделения коллокаций существуют разные метрики. Мы познакомимся с одной из них, которая называется **[Pointwise Mutual Information (PMI)](https://en.wikipedia.org/wiki/Pointwise_mutual_information)**. Формула рассчета PMI - 

$$ pmi = log \frac{{p(x,y)}}{p(x) \cdot p(y)} $$

* x и y - слова, входящие в биграмму;
* p(x,y) - вероятность появления биграммы x + y;
* p(x) и p(y) -- вероятность появления каждого из элементов биграммы в отдельности.

Значит, чтобы посчитать PMI, нам нужно:

* взять текст
* сделать частотный список всех слов в тексте
* найти вероятность появления каждого слова в тексте, разделив его частотность на общее количество слов
* сделать частотный список всех биграмм в тексте
* найти вероятность появления каждой биграммы в тексте, разделив её частотность на общее количество биграмм (а сколько их?)
* найти pmi по формуле
* биграммы с наибольшим pmi - коллокации в этом тексте.

Давайте попробуем:

In [1]:
import re
from math import log

punct = '[.,!«»?&@"$\[\]\(\):;%#&\'—-]'

def preprocessing(text):
    text_wo_punct = re.sub(punct, '', text.lower())
    words = text_wo_punct.strip().split()
    return words

In [2]:
with open('../news.txt', 'r', encoding='utf-8') as f:
    words = preprocessing(f.read())

word_freq = {}
for word in words:
    if word in word_freq:
        word_freq[word] += 1
    else:
        word_freq[word] = 1

bigrams = []
for ind in range(1, len(words) - 1):
    bigrams.append(' '.join([words[ind - 1], words[ind]]))
    
bigram_freq = {}
for b in bigrams:
    if b in bigram_freq:
        bigram_freq[b] += 1
    else:
        bigram_freq[b] = 1

In [3]:
def count_pmi(x, y):
    p_xy = bigram_freq[' '.join([x, y])]/len(bigram_freq)
    p_x, p_y = word_freq[x]/len(word_freq), word_freq[y]/len(word_freq)
    pmi = log(p_xy/(p_x * p_y))
    return pmi

In [4]:
pmi = {}
for bigr in bigrams:
    x, y = bigr.split()
    pmi[bigr] = count_pmi(x, y)

i = 0
for bigram in sorted(pmi, key = lambda m: -pmi[m]):
    if i > 100:
        break
    print(bigram, pmi[bigram])
    i += 1

шотландских националистов 8.766034796769167
удовлетворении ходатайства 8.766034796769167
оренбургскими авиа­линиями 8.766034796769167
луганский тепловозостроительный 8.766034796769167
sukhoj superjet100 8.766034796769167
вызвавшие возражение 8.766034796769167
narrative about 8.766034796769167
султан батов 8.766034796769167
портовому дивизиону 8.766034796769167
блокады поручили 8.766034796769167
аналитическую записку 8.766034796769167
школьницы танцевали 8.766034796769167
шоколада сыра 8.766034796769167
доставлять наличную 8.766034796769167
рудник удачный 8.766034796769167
ngsru представленная 8.766034796769167
филип бридлав 8.766034796769167
салианн скорпуллу 8.766034796769167
депонировать причитающиеся 8.766034796769167
бориса ротенбергов 8.766034796769167
нигилистически относящаяся 8.766034796769167
страницы mailru 8.766034796769167
московскую кольцевую 8.766034796769167
очистке стоков 8.766034796769167
владению пользованию 8.766034796769167
участковый милиционер 8.766034796769167
вс

Какие слова имеют большой коэффициент pmi, а какие маленький?

С помощью pmi можно также искать биграммы, специфичные для какой-то категории текстов. Для этого нам нужно всего лишь посчитать коэффициент PMI для слова/биграммы и категории.

Например, у нас есть тексты 3 категорий. Посчитаем для слов в этих текстах коэффициент PMI, при этом в формуле X будет словом, а Y - категорией. (X, Y) - сколько раз слово X встретилось в текстах категории Y.

In [10]:
import os
anek = ''
teh = ''
izvest = ''
for root, dirs, files in os.walk('texts'):
    for f in files:
        if 'anekdots' in root:
            num_anek = len(files)
            anek += open(os.path.join(root, f)).read()
        elif 'izvest' in root:
            num_izvest = len(files)
            izvest += open(os.path.join(root, f)).read()
        elif 'teh_mol' in root:
            num_teh = len(files)
            teh += open(os.path.join(root, f)).read()
            
words_anek = preprocessing(anek)
words_teh = preprocessing(teh)
words_izvest = preprocessing(izvest)

words = words_anek + words_teh + words_izvest

In [11]:
def freq_dict(arr):
    dic = {}
    for element in arr:
        if element in dic:
            dic[element] += 1
        else:
            dic[element] = 1
    return dic

In [12]:
corpus_freq = freq_dict(words)
anek_freq = freq_dict(words_anek)
izvest_freq = freq_dict(words_izvest)
teh_freq = freq_dict(words_teh)

In [13]:
def pmi_for_cats(x, y):
    if y == 'anek':
        dic = anek_freq
        num = num_anek
    elif y == 'teh':
        dic = teh_freq
        num = num_teh
    elif y == 'izvest':
        dic = izvest_freq
        num = num_izvest
    p_xy = dic[x]/len(dic)
    p_x, p_y = corpus_freq[x]/len(corpus_freq), num/(num_izvest + num_teh + num_anek)
    pmi = log(p_xy/(p_x * p_y))
    return pmi

In [14]:
cat_pmi = {}
i = 0
for word in corpus_freq:
    if i > 100:
        break
    try:
        pmi_anek = pmi_for_cats(word, 'anek')
    except KeyError:
        pmi_anek = 0
    try:
        pmi_teh = pmi_for_cats(word, 'teh')
    except KeyError:
        pmi_teh = 0
    try:
        pmi_izvest = pmi_for_cats(word, 'izvest')
    except KeyError:
        pmi_izvest = 0
    max_pmi = max(pmi_anek, pmi_teh, pmi_izvest)
    if max_pmi == 0:
        continue
    if max_pmi == pmi_anek:
        cat = 'anek'
    elif max_pmi == pmi_teh:
        cat = 'teh'
    elif max_pmi == pmi_izvest:
        cat = 'izvest'
    print(word, cat)
    i += 1

трансурановый anek
устанавливает izvest
набежали anek
атомном anek
раздельного teh
кровоток izvest
возникает teh
гайдар anek
выставления izvest
потерпевшему anek
цвету anek
постового anek
чэм anek
бедно anek
лили anek
вереи teh
уборки teh
души izvest
выкладки teh
сделалто anek
остановок anek
беседу anek
comdex izvest
прикинуться anek
осужденных izvest
athlon izvest
вввам anek
высвобождается teh
интерферона teh
всплыла anek
глазастенький anek
голые anek
львица izvest
отряхивает anek
гранат anek
проверка teh
значение teh
хорошей anek
информирования izvest
рамн izvest
подзарядка teh
сферической teh
сжег anek
интернетбизнеса izvest
бесстрастнонеприступная teh
реальный izvest
взволнованно anek
изготавливали teh
привела anek
штабной teh
заполняющих teh
дома anek
руках anek
овацию anek
крепят anek
раздосадованно anek
бедняфка anek
построенная teh
порту anek
развитием izvest
совокупляются anek
дубовый anek
натуральную teh
логичную izvest
относишься anek
пххх anek
разбогатеть anek
трагедией teh

## Задания

1. Отфильтровать слова, для которых мы вычисляем категории: убрать короткие и служебные, взять самые частотные.
2. Соотнести биграммы с категориями.